## Lexical Parser and Analyzer
CS242 Project - Simple Calculator


**TODO**
Identifiers:
    - Print
    - if (conditional)
    - sqrt
    - relational (>, >=, <, <=, ==, !=)


In [334]:
letters = ["A", "B", "C", "D", "E", "F",
   "G", "H", "I", "J", "K", "L",
   "M", "N", "O", "P", "Q", "R", 
   "S", "T", "U", "V", "W", "X", 
   "Y", "Z"] 

digits = ["0", "1", "2", "3", "4", 
  "5", "6", "7", "8", "9"]

tf = {0: {"letter": 1, "space": 0, "e": 1, "quotation": 3, "equals": 5, "plus": 6, 
          "minus": 7, "mult":8, "slash":11, "modulo": 12, "lparen": 13, 
          "rparen": 14, "comma": 15, "semicolon": 16, "eof": 17, "dot": 26,
          "other": 0, "nl":0, "digit": 18, "pound": 24, "greater":27, "lesser":28},
      # Identifier
      1: {"letter": 1, "e": 1, "digit": 1, "space": 2, "equals": 2, "nl":2, 
          "lparen": 2, "rparen": 2, "semicolon": 2, "eof": 2, "dot": 2,
          "slash": 2, "modulo": 2, "mult": 2, "plus": 2, "minus": 2, "comma":2,
          "error": 2},
      # String
      3: {"letter": 3, "e": 3, "digit": 3, "space": 3, "nl":3, 
          "dot": 3, "lparen": 3, "rparen": 3, "slash": 3, "modulo": 3, 
          "plus": 3, "minus": 3, "equal": 3, "comma": 3,
          "semicolon": 3, "mult": 3, "other":3, "quotation": 4,
          "error": 3}, 
      # Multiply
      8: {"letter": 9, "e": 9, "digit": 9, "space": 9, "lparen": 9, "rparen": 9, 
          "mult": 10, "minus": 9, "error": 2},      
      # Slash
      11: {"letter": 23, "e": 23, "digit": 23, "space": 23, "lparen": 23, 
           "rparen": 23, "slash": 24, "minus":23, "error": 2},
      # Comment
      24: {"letter": 24, "e": 24, "digit": 24, "other": 24, "eof": 24,
          "dot": 24, "lparen": 24, "rparen": 24, "modulo": 24, "space": 24,
          "plus": 24, "minus": 24, "equals": 24, "quotation": 24, "comma": 24,
          "semicolon": 24, "eof": 24, "mult": 24, "slash": 24, "nl":25},
      # Digit
      18: {"digit": 18, "dot": 26, "e": 20, "space": 22, "nl":22, "lparen": 22, 
           "rparen": 22, "slash": 22, "modulo": 22, "plus": 22, "mult": 22,
           "minus": 22, "equal": 22, "comma": 22, "semicolon": 22, 
           "eof": 22, "error": 1},
      # Digit with no dot after
      19: {"digit": 19, "e": 20, "space":22, "nl":22, "lparen": 22, "rparen": 22, 
           "slash": 22, "modulo": 22, "mult": 22, "plus": 22, "minus": 22, 
           "equals": 22, "comma": 22, "semicolon": 22, "eof": 22, "error": 1},
      # Exponent
      20: {"plus": 21, "minus": 21, "digit": 21, "error": 1}, 
      # Digit after exponent
      21: {"digit": 21, "e": 22, "space":22, "nl":22, "lparen": 22, "rparen": 22, 
           "slash": 22, "modulo": 22, "plus": 22, "minus": 22, 
           "equal": 22, "quotation": 22, "comma": 22,
           "semicolon": 22, "eof": 22, "mult": 22, "error": 1},
      26: {"digit": 19, "error":1},
     }

accept_states = {2: "IDENT", 4: "STRING", 5: "EQUALS", 6: "PLUS", 7: "MINUS", 9: "MULT", 
                 10: "EXP", 23:"DIVIDE", 12: "MODULO", 13: "LPAREN", 14: "RPAREN", 15: "COMMA",
                 16: "SEMICOL", 17: "EOF", 22: "NUMBER", 25: "COMMENT", 27: "GREATER", 28: "LESSER"}

errors = {1: "Badly-formed number", 2: "Illegal Character", 3: "Unterminated String"}

In [215]:
class LexAnalyzer:
    def __init__(self, text=None, fn=None):
        if fn != None:
            self.input = open(fn).read()
        else:
            self.input = text
        
        self.input = self.input.upper()
        self.position = 0
        
        self.start_state = 0
        self.current_state = 0
        
        self.token = self.input[self.position]
        self.token_type = ""
        
        self.token_str = ""
        self.lexeme = ""
        
        self.err_flag = False
        self.output_ready = False
        self.terminate = False

    def setState(self, state):
        self.current_state = state   
        
    def getNextToken(self):
        if self.position == len(self.input) -1:
            self.token = '$' # end of string marker
        else:
            self.position += 1
            self.token = self.input[self.position]
        
    def check_token_type(self, token):
        token_type_dict = {'"': "quotation", "=": "equals", "+": "plus", "-": "minus",
                           "*": "mult", "/": "slash", "%": "modulo", "(": "lparen",
                           ")": "rparen", ",": "comma", ";": "semicolon", "$": "eof", 
                           ".": "dot", " ": "space", ">": "greater", "<": "lesser",
                           "\n": "nl", "E": "e", "#": "pound"}
        
        if token in token_type_dict.keys():
            return token_type_dict[token]
        
        elif token in letters:
            return "letter"
        elif token in digits:
            return "digit"
        else:
            return "other"
    
    def transitionState(self):
        self.token_type = self.check_token_type(self.token)

        # Match to tf, update state  
        if self.token_type in tf[self.current_state].keys():
            self.setState(tf[self.current_state][self.token_type])
        else:
            self.err_flag = True            

        # When state not start state, start saving self.lexeme
        if self.current_state != self.start_state:
            self.lexeme += self.token

        # If state in accept state, print TOKEN and saved self.lexeme, then clear self.lexeme var and return to start state
        if self.current_state in accept_states.keys():    
            self.token_str = accept_states[self.current_state]
            if self.current_state in [2, 9, 22, 23]: # those with other required before accepting state 
                self.lexeme = self.lexeme[:-1]
                if self.lexeme in ["PRINT", "IF", "SQRT"]:
                    self.token_str = self.lexeme
                self.position -= 1
            
            # If state is comment, just empty lexeme. If not, print output 
            if self.current_state == 25:   
                self.lexeme = ""    
            else:
                self.output_ready = True
            
            self.setState(self.start_state)
            
        # Break loop if eof and error detected, else get next token and continue
        if self.token == "$":
            self.terminate = True

        elif self.err_flag == True:
            print("Current State: {}. Token not recognized: {}".format(self.current_state, self.token))
            print("ERROR: {}".format(errors[tf[self.current_state]["error"]]))
            self.terminate = True

**CFG**
S -> IF(COND) E; | IF(COND) S; | PRINT(STRING); | IDENT = E;

COND -> E REL E 
REL -> '<' | '>' | '<=' | '>=' | '==' | '!='

E -> E + EA | E - EA | EA 
EA -> EA * EB | EA / EB | EB
EB -> -EC | EC
EC -> EC * * ED | ED

ED -> NUMBER | IDENT | ( E ) | SQRT(E)

**LL1**

ex: what = 10+2;
S -> IF(COND) E SEMICOL | IF(COND) S SEMICOL | PRINT(STRING) SEMICOL | IDENT = E SEMICOL

COND -> E REL E 
REL -> '<' | '>' | '<=' | '>=' | '==' | '!='

E -> EA E' 
E' -> + EA E' | - EA E' | epsilon

EA -> EB EA' 
EA' -> * EB EA' | / EB EA' | epsilon

EB -> -EC | EC

EC -> ED EC' 
EC' -> * * ED EC' | ED | epsilon

ED -> NUMBER | IDENT | ( E ) | SQRT(E)

In [335]:
#TODO replicate doc v's playwithones
class SynAnalyzer:
    def __init__(self, lex_pairs=None):
        self.lex_pairs = lex_pairs
        self.position = -1;
        self.token = ""
        self.lexeme = ""
        self.variables = {}
        
        self.err_flag = False
        
        self.rel = ['<', '>', '<=', '>=', '==', '!=']
        self.E_ident = ["NUMBER", "IDENT", "LPAREN", "RPAREN", "SQRT"]
    
    def set_next_pair(self):
        if self.token == "EOF":
            print("End reached.")
        else:
            self.position = self.position + 1
            self.token = self.lex_pairs[self.position][0]
            self.lexeme = self.lex_pairs[self.position][1]
            
    def get_variable_value(self, var):
        dict_var = dict(self.variables)
        if var in dict_var.keys():
            return dict_var[var]
        return None
        
    def error(self, error_type, class_src):
        errors = {1: "Unexpected token", 2: "Inappropriate argument"}
        print("[{}] Error: {}; Token: {}; Lexeme: {}; Position: {}".format(class_src, errors[error_type], 
                                                               self.token, self.lexeme, self.position))
        self.err_flag = True
    
    def check_semicolon(self):
        if self.token != "SEMICOL":
            self.err_flag = True
            self.err_msg = "Missing semicolon"
            return False
        elif self.token == "SEMICOL" and self.token != "EOF":
            print("Done statement.")
            self.set_next_pair()
            self.S()
        return True
            
    # Production S -> IF(COND) E ; | IF(COND) S | PRINT(STRING) ; | PRINT(IDENT) ; | IDENT = E ;
    def S(self): 
        res = -1
        ans = None
        if self.token == "IF":
            self.set_next_pair()
            if self.token == "LPAREN":
                self.set_next_pair()
                if self.token in self.E_ident:
                    cond_res = self.COND() 
                    self.set_next_pair()
                    if self.token == "RPAREN":
                        self.set_next_pair()
                        if cond_res:
                            print("Condition met.", end=" ")
                            
                            # Handling expression
                            if self.token in self.E_ident:
                                ans = self.E()
                                print("Computation performed", ends=" ") 
                                self.set_next_pair()
                                if not self.check_semicolon():
                                    self.error(1, "S")
                                
                                    
                            # Recurse to S
                            elif self.token in ["IF", "PRINT", "IDENT"]: # S identifiers
                                self.S()
                        else:
                            print("Condition not met.")
                    else:
                        self.error(1, "S")
                else:
                    self.error(1, "S")
            else:
                self.error(1, "S")
                
            return ans

        elif self.token == "PRINT":
            msg = ""
            self.set_next_pair()
            
            # check format (STRING) or (IDENT)
            if self.token == "LPAREN":
                self.set_next_pair()
                if self.token == "STRING":
                    msg = self.lexeme
                    
                elif self.token == "IDENT":
                    msg = self.get_variable_value(self.lexeme)
                    if msg is None:
                        self.error("undeclared variable")
                else:
                    self.error(2, "S")
                            
                
                self.set_next_pair()
                if self.token == "RPAREN":
                    self.set_next_pair()
                    if self.check_semicolon():
                        print("Output: ({})".format(msg))
                else:
                    self.error(1, "S")
            else:
                self.error(1, "S")
                
            return True
            
        elif self.token == "IDENT":
            var_name = self.lexeme
            self.set_next_pair()
            
            # Assignment procedure
            if self.token == 'EQUALS':  
                self.set_next_pair()
                if self.token in self.E_ident:
                    val = self.E()
                    self.variables[var_name] = val
                    print("Computation performed ({} = {})".format(var_name, val))
                    if not self.check_semicolon():
                        self.error(1, "S")
                else:
                    self.error(1, "S")
            else:
                self.error(1, "S")
                            
            return True
        
    # should return true or false
    def COND(self):
        return 0

    # Production E -> EA E' 
    # should return value
    def E(self):
        print("In E: Lexeme: {}".format(self.lexeme))
        x = self.EA()
        y = self.E_prime()
        print("In E: {}, {}".format(x,y))
        return x + y

    # Production E' -> + EA E' | - EA E' | epsilon
    def E_prime(self):
        print("In E_PRIME: Lexeme: {}".format(self.lexeme))
        if self.token == "PLUS":
            self.set_next_pair()
            x = self.EA()
            y = self.E_prime()
            print("In E_PRIME: {} + {}".format(x,y))
            return x + y
        elif self.token == "MINUS":
            self.set_next_pair()
            x = self.EA()
            y = self.E_prime()
            print("In E_PRIME: {} - {}".format(x,y))
            return -x + y
        else:
            return 0

    # Production EA -> EB EA' 
    def EA(self):
        x = self.EB()
        y = self.EA_prime()
        print("In EA: {}, {}".format(x,y))
        return x * y
    
    # Production EA' -> * EB EA' | / EB EA' | epsilon
    def EA_prime(self):
        if self.token == "MULT":
            self.set_next_pair()
            x = self.EB()
            y = self.EA_prime()
            return x * y
        elif self.token == "DIVIDE":
            self.set_next_pair()
            x = self.EB()
            y = self.EA_prime()
            return 1/(x * y)
        else:
            return 1

    # Production EB -> -EC | EC
    def EB(self):
        print(self.token)
        if self.token == "MINUS":
            self.set_next_pair()
            return -1*self.EC()
        else:
            return self.EC()
    
    # Production EC -> ED EC' 
    def EC(self):
        x = self.ED()
        y = self.EC_prime()
        print("In EC: {}, {}".format(x,y))
        return x ** y


    # Production EC' -> * * ED EC' | epsilon
    def EC_prime(self):
        if self.token == "EXP":
            self.set_next_pair()
            x = self.ED()
            y = self.EC_prime()
            print("{}, {}".format(x,y))
            return x * y
        else:
            return 1


    # Production ED -> NUMBER | IDENT | (E) | SQRT(E)
    def ED(self):
        if self.token == "NUMBER":
            val = int(self.lexeme)
            self.set_next_pair()
            return val
        elif self.token == "IDENT":
            val = get_variable_value(self.lexeme)
            self.set_next_pair()
            return val
        elif self.token == "LPAREN":
            self.set_next_pair()
            val = self.E()
            self.set_next_pair()
            if self.token == "RPAREN":
                self.set_next_pair()
                return val
            else:
                self.error(1, "ED")
        elif self.token == "SQRT":
            self.set_next_pair()
            if self.token == "LPAREN":
                self.set_next_pair()
                val = self.E()
                self.set_next_pair()
                if self.token == "RPAREN":
                    self.set_next_pair()
                    return val ** (0.5)
                else:
                    self.error(1, "ED")
            else:
                self.error(1, "ED")
        else:
            self.error(1, "ED")
        return False


In [336]:
def main():
    la = LexAnalyzer(fn="/home/fassster/Documents/cs242/proj1/samples/files/prog1.txt")
    debug_print = True
    
    print("=====================================")
    print("INPUT (LENGTH: {})".format(len(la.input)))
    print("=====================================")
    print("{}\n".format(la.input))
    
    print("=====================================")
    print("RESULT")
    print("=====================================\n")
    
    if debug_print:
        print("TOKEN | LEXEME")
        print("---------------")
    
    # [(token, lexeme), (), ()]
    pairs = []
    while True:
        
        la.transitionState()
        
        if la.output_ready:
            pair = (la.token_str, la.lexeme)
            pairs.append(pair)
            if debug_print:
                print("{}: {}".format(pair[0], pair[1]))
            
            la.output_ready = False
            la.lexeme = ""
             
        la.getNextToken() 
        
        if la.terminate:
            if not la.err_flag:
                print("\n=> LEXICAL ANALYSIS: SUCCESSFUL\n")
            break
           
    if not la.err_flag:
        sa = SynAnalyzer(lex_pairs=pairs)
        sa.set_next_pair()
        print(sa.S())
        print(sa.variables)
        
main()

INPUT (LENGTH: 132)
// THIS PROGRAM CALCULATES THE ROOTS OF A QUADRATIC EQUATION
WHAT = 10+2-8*4;
HA = 2*6;
SL = 12/7;
LOL = 4**3*2/4; 
HEY = 36-3/-2;



RESULT

TOKEN | LEXEME
---------------
IDENT: WHAT
EQUALS: =
NUMBER: 10
PLUS: +
NUMBER: 2
MINUS: -
NUMBER: 8
MULT: *
NUMBER: 4
SEMICOL: ;
IDENT: HA
EQUALS: =
NUMBER: 2
MULT: *
NUMBER: 6
SEMICOL: ;
IDENT: SL
EQUALS: =
NUMBER: 12
DIVIDE: /
NUMBER: 7
SEMICOL: ;
IDENT: LOL
EQUALS: =
NUMBER: 4
EXP: **
NUMBER: 3
MULT: *
NUMBER: 2
DIVIDE: /
NUMBER: 4
SEMICOL: ;
IDENT: HEY
EQUALS: =
NUMBER: 36
MINUS: -
NUMBER: 3
DIVIDE: /
MINUS: -
NUMBER: 2
SEMICOL: ;
EOF: $

=> LEXICAL ANALYSIS: SUCCESSFUL

In E: Lexeme: 10
NUMBER
In EC: 10, 1
In EA: 10, 1
In E_PRIME: Lexeme: +
NUMBER
In EC: 2, 1
In EA: 2, 1
In E_PRIME: Lexeme: -
NUMBER
In EC: 8, 1
NUMBER
In EC: 4, 1
In EA: 8, 4
In E_PRIME: Lexeme: ;
In E_PRIME: 32 - 0
In E_PRIME: 2 + -32
In E: 10, -30
Computation performed (WHAT = -20)
Done statement.
In E: Lexeme: 2
NUMBER
In EC: 2, 1
NUMBER
In EC: 6, 1
In

In [130]:
int("10") + 1


11